In [1]:
import glob
import os
import cv2
import numpy as np
import shutil
from matplotlib import pyplot as plt
from skimage.filters import threshold_otsu
from sklearn.cluster import MeanShift
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import matplotlib.patches as patches

In [2]:
def bb_intersection_over_union(boxA, xBmin, yBmin, xBmax, yBmax):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = np.maximum(boxA[0], xBmin)
    yA = np.maximum(boxA[1], yBmin)
    xB = np.minimum(boxA[2], xBmax)
    yB = np.minimum(boxA[3], yBmax)
 
    # compute the area of intersection rectangle
    interArea = np.maximum(0, xB - xA + 1) * np.maximum(0, yB - yA + 1)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (xBmax - xBmin + 1) * (yBmax - yBmin + 1)
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = np.divide(interArea, (boxBArea + boxAArea - interArea).astype(np.float))

    # return the intersection over union value
    return iou

In [3]:
def merge_bb(boxes):
    pick = [0, 0, 0, 0]
    boxes = boxes.astype("float")
    # convert (w,h) to (x2,y2)
    boxes[:,2] = boxes[:,0] + boxes[:,2]
    boxes[:,3] = boxes[:,1] + boxes[:,3]
    counter = 0
    while counter < len(boxes):
        iou = bb_intersection_over_union(boxes[counter,:],boxes[:,0],boxes[:,1],boxes[:,2],boxes[:,3])
        if len(boxes[(iou>0),:]) > 1:
            merging = boxes[(iou>0),:]
            xmin = min(merging[:,0])
            ymin = min(merging[:,1])
            xmax = max(merging[:,2])
            ymax = max(merging[:,3])
            pick = np.vstack([pick, [xmin,ymin,xmax,ymax]])
        else:
            merging = boxes[(iou>0),:]
            pick = np.vstack([pick, merging])            
        counter = counter + 1
    pick[:,2] = pick[:,2] - pick[:,0]
    pick[:,3] = pick[:,3] - pick[:,1]
    return pick[1:,:]

In [4]:
def non_max_suppression_fast(boxes, overlapThresh):
   # if there are no boxes, return an empty list
   if len(boxes) == 0:
      return []
 
   # initialize the list of picked indexes   
   pick = []

   # grab the coordinates of the bounding boxes
   x = boxes[:,0].astype("float")
   y = boxes[:,1].astype("float")
   w = boxes[:,2].astype("float")
   h = boxes[:,3].astype("float")

   x1 = x - w/2
   y1 = y - h/2
   x2 = x + w/2
   y2 = y + h/2

   # compute the area of the bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
   area = (w + 1) * (h + 1)
   idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
   while len(idxs) > 0:
      # grab the last index in the indexes list and add the
      # index value to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of
      # the bounding box and the smallest (x, y) coordinates
      # for the end of the bounding box
      xx1 = np.maximum(x1[i], x1[idxs[:last]])
      yy1 = np.maximum(y1[i], y1[idxs[:last]])
      xx2 = np.minimum(x2[i], x2[idxs[:last]])
      yy2 = np.minimum(y2[i], y2[idxs[:last]])

      # compute the width and height of the bounding box
      w = np.maximum(0, xx2 - xx1 + 1)
      h = np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have
      idxs = np.delete(idxs, np.concatenate(([last],
         np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked using the
   # integer data type
   return boxes[pick].astype("int")

### Loop

In [5]:
for foldername in glob.glob("clusters_training/*"):
    #print(foldername)
    for filename in glob.glob(foldername+"/*.jpg"):
        fn = filename.split('/')
        fn = fn[-1:][0][:-4]
        img_to_mser = cv2.imread(filename,0)
        img_to_crop = cv2.imread(filename)
        mser = cv2.MSER_create(_min_area=10)
        coordinates, bboxes = mser.detectRegions(img_to_mser)
        area = (abs((bboxes[:,0] - bboxes[:,2])*(bboxes[:,1] - bboxes[:,3])).astype(float))/(img_to_crop.shape[0]*img_to_crop.shape[1])
        # remove outliers
        mean = np.mean(bboxes[:,3],axis=0)
        sd = np.std(bboxes[:,3],axis=0)
        bboxes = bboxes[(bboxes[:,3] < mean + 2*sd) & (bboxes[:,3] < np.percentile(bboxes[:,3], 75)), :]
        # when the bounding box is small add width onto.
        bias = 6
        wlen = 6
        bboxes[bboxes[:,2] < wlen,0] = bboxes[bboxes[:,2] < wlen,0] - bias/2
        bboxes[bboxes[:,2] < wlen,2] = bboxes[bboxes[:,2] < wlen,2] + bias
        bboxes = merge_bb(bboxes)
        bboxes = non_max_suppression_fast(bboxes,0.7)
        #fig,ax = plt.subplots(1)
        #ax.imshow(img_to_crop)
        j = 0
        for bbox in np.unique(bboxes,axis=0):
            x, y, w, h = bbox.astype(int)
            w = w
            h = h
            x = x
            y = y
            if w > h*(1.5) and h > 10 and h < 25 and w < 6*h:
                #rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
                # Add the patch to the Axes
                #ax.add_patch(rect)
                wcrop = img_to_crop[y:y+h,x:x+w]
                cv2.imwrite("unsupervised_census_docs/"+fn+"_"+str(j).zfill(3)+".jpg",wcrop)
            j = j + 1
        #plt.show()